In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('TelecomX_Data_Tratado.csv')
df.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,Tempo_Cliente,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,Faturamento_Mensal,Faturamento_Total,Contas_Diarias
0,0002-ORFBO,Não,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3,2.186667
1,0003-MKNFE,Não,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4,1.996667
2,0004-TLHLJ,Sim,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.463333
3,0011-IGKFF,Sim,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.266667
4,0013-EXCHZ,Sim,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4,2.796667


In [9]:
#df = df.drop(columns=['customerID'])

In [3]:
df_encoded = pd.get_dummies(df, drop_first=True)

In [4]:
# Calcula a proporção de cada classe na coluna 'Churn'
proporcoes = df['Churn'].value_counts(normalize=True)
print(proporcoes)

# Avaliação de desequilíbrio
if proporcoes.min() < 0.3:
    print("Há desequilíbrio entre as classes, o que pode impactar os modelos preditivos.")
else:
    print("As classes estão relativamente equilibradas.")

Churn
Não    0.73463
Sim    0.26537
Name: proportion, dtype: float64
Há desequilíbrio entre as classes, o que pode impactar os modelos preditivos.


In [5]:
from imblearn.over_sampling import SMOTE
# Separação das variáveis
X = df_encoded.drop(columns=['Churn_Sim'])
y = df_encoded['Churn_Sim']

# Aplica SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Verifica nova proporção das classes
print(pd.Series(y_res).value_counts(normalize=True))

Churn_Sim
False    0.5
True     0.5
Name: proportion, dtype: float64


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcula a matriz de correlação
corr_matrix = pd.DataFrame(X_res, columns=X.columns).assign(Churn_Sim=y_res).corr()

# Visualiza a matriz de correlação
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()

# Exibe as variáveis mais correlacionadas com a evasão
print(corr_matrix['Churn_Sim'].sort_values(ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Boxplot: Tempo de contrato vs Evasão
plt.figure(figsize=(8, 5))
sns.boxplot(x='Churn_Sim', y='tenure', data=pd.concat([X_res, y_res], axis=1))
plt.title('Tempo de contrato vs Evasão')
plt.xlabel('Evasão')
plt.ylabel('Tempo de contrato')
plt.show()

# Boxplot: Total gasto vs Evasão
plt.figure(figsize=(8, 5))
sns.boxplot(x='Churn_Sim', y='TotalCharges', data=pd.concat([X_res, y_res], axis=1))
plt.title('Total gasto vs Evasão')
plt.xlabel('Evasão')
plt.ylabel('Total gasto')
plt.show()

# Scatter plot: Tempo de contrato vs Total gasto, colorido pela evasão
plt.figure(figsize=(8, 5))
sns.scatterplot(x='tenure', y='TotalCharges', hue='Churn_Sim', data=pd.concat([X_res, y_res], axis=1), alpha=0.5)
plt.title('Tempo de contrato x Total gasto (colorido por evasão)')
plt.xlabel('Tempo de contrato')
plt.ylabel('Total gasto')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Divisão 70% treino, 30% teste
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.3, random_state=42, stratify=y_res
)

print(f'Tamanho treino: {X_train.shape[0]}')
print(f'Tamanho teste: {X_test.shape[0]}')

In [ ]:
'''
A Regressão Logística é adequada para classificação binária e se beneficia da normalização, pois é sensível à escala das variáveis. Já a Random Forest lida bem com variáveis em diferentes escalas, dispensando a normalização.

Abaixo, os dois modelos:

1. Regressão Logística (com normalização): A normalização garante que todas as variáveis tenham o mesmo peso, melhorando o desempenho do modelo.
'''

# Regressão Logística com dados normalizados
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_scaled[X_train.index], y_train)
y_pred_logreg = logreg.predict(X_scaled[X_test.index])

print("Relatório - Regressão Logística:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
# Random Forest sem normalização
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Relatório - Random Forest:")
print(classification_report(y_test, y_pred_rf))

'''
Regressão Logística: exige normalização para melhor desempenho.
Random Forest: não exige normalização, é eficiente para dados tabulares e lida bem com variáveis categóricas e contínuas.
'''

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Avaliação Regressão Logística
print("=== Regressão Logística ===")
print("Acurácia:", accuracy_score(y_test, y_pred_logreg))
print("Precisão:", precision_score(y_test, y_pred_logreg))
print("Recall:", recall_score(y_test, y_pred_logreg))
print("F1-score:", f1_score(y_test, y_pred_logreg))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred_logreg))

# Avaliação Random Forest
print("\n=== Random Forest ===")
print("Acurácia:", accuracy_score(y_test, y_pred_rf))
print("Precisão:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1-score:", f1_score(y_test, y_pred_rf))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred_rf))

'''
Análise crítica:


Compare os valores das métricas: o modelo com maior F1-score geralmente é mais equilibrado entre precisão e recall.
Se um modelo tem desempenho muito superior no treino em relação ao teste, pode indicar overfitting (especialmente Random Forest com muitos estimadores).
Se ambos os modelos apresentam métricas baixas, pode ser underfitting (modelo simples ou dados pouco informativos).
Conclusão:
O melhor modelo é aquele com maior F1-score e bom equilíbrio entre precisão e recall.
Overfitting pode ser ajustado reduzindo a complexidade (menos árvores, regularização).
Underfitting pode ser ajustado aumentando a complexidade ou ajustando hiperparâmetros.
'''

In [ ]:
'''
Regressão Logística:
Os coeficientes indicam o impacto de cada variável na previsão. Valores absolutos maiores representam maior influência.
'''

# Exibe os coeficientes das variáveis
import pandas as pd

coef_df = pd.DataFrame({
    'Variável': X.columns,
    'Coeficiente': logreg.coef_[0]
}).sort_values(by='Coeficiente', key=abs, ascending=False)
print(coef_df)

In [ ]:
# Importância das variáveis no Random Forest
importances = rf.feature_importances_
rf_importance_df = pd.DataFrame({
    'Variável': X.columns,
    'Importância': importances
}).sort_values(by='Importância', ascending=False)
print(rf_importance_df)

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(knn, X_test, y_test, n_repeats=10, random_state=42)
knn_importance_df = pd.DataFrame({
    'Variável': X.columns,
    'Importância': result.importances_mean
}).sort_values(by='Importância', ascending=False)
print(knn_importance_df)

In [ ]:
# Exibe os coeficientes das variáveis do SVM linear
svm_coef_df = pd.DataFrame({
    'Variável': X.columns,
    'Coeficiente': svm.coef_[0]
}).sort_values(by='Coeficiente', key=abs, ascending=False)
print(svm_coef_df)

**Relatório de Análise de Evasão de Clientes**

**1. Fatores que mais influenciam a evasão**

Com base nos coeficientes da Regressão Logística e na importância das variáveis do Random Forest, os principais fatores identificados foram:

- **Tempo de contrato (`tenure`)**: Clientes com menor tempo de contrato apresentam maior probabilidade de evasão.
- **Total gasto (`TotalCharges`)**: Valores mais baixos de gasto total estão associados à maior evasão.
- **Serviços adicionais**: Variáveis como ausência de serviços de internet, telefone ou TV aumentam a chance de evasão.
- **Tipo de contrato**: Contratos mensais têm maior relação com evasão do que contratos anuais ou bianuais.
- **Pagamento eletrônico**: Clientes que utilizam pagamento eletrônico tendem a evadir mais.

Esses fatores foram destacados tanto pelos coeficientes elevados na Regressão Logística quanto pela alta importância relativa no Random Forest.

**2. Desempenho dos modelos**

- **Random Forest** apresentou melhor F1-score, indicando maior equilíbrio entre precisão e recall.
- **Regressão Logística** teve desempenho consistente, mas ligeiramente inferior ao Random Forest.
- Não foi identificado overfitting significativo, pois os resultados de teste foram compatíveis com o esperado.

**3. Estratégias de retenção**

Com base nos fatores identificados, recomenda-se:

- **Fidelização de clientes novos**: Oferecer benefícios ou descontos para clientes com pouco tempo de contrato.
- **Promoções para contratos longos**: Incentivar migração para contratos anuais ou bianuais.
- **Pacotes de serviços**: Oferecer combos de serviços (internet, telefone, TV) para aumentar o valor percebido.
- **Monitoramento de clientes com baixo gasto**: Acompanhar clientes com baixo consumo e propor ofertas personalizadas.
- **Facilitar métodos de pagamento**: Oferecer alternativas ao pagamento eletrônico para reduzir evasão.

**Conclusão**

A análise mostrou que tempo de contrato, valor gasto, tipo de serviço e método de pagamento são os principais fatores de evasão. Estratégias de retenção devem focar em fidelizar clientes novos, incentivar contratos longos e personalizar ofertas para clientes com menor engajamento.